# Class Exercises - Week 3

---
---

**Author:** Dr Giordano Scarciotti (g.scarciotti@imperial.ac.uk) - Imperial College London

**Module:** ELEC70066 - Applied Advanced Optimisation

**Version:** 1.1.1 - 23/01/2025

---
---

**Reminder:** The solutions of some these exercises may be found online. Do not look for the solutions, but ask for help to me or the GTA. Your assessment will not have a solution available online so it is of paramount importance that you learn how to solve the problems without relying on a solution being available.

The solutions of these exercises will be discussed at the end of the session. Moreover, the complete solution will be provided to you after 1 week. **N.B.** **Do not share the solutions online**

## Exercise 1

Two investments are made, with random returns $R_1$ and $R_2$. The total return for the two investments is $R_1 + R_2$, and the probability of a loss (including breaking even, i.e., $R_1+R_2=0$) is $p^{loss} = \textbf{prob}(R_1 + R_2 \le 0)$. The goal is to find the worst-case (i.e., maximum possible) value of $p^{loss}$ consistent with the following information.

We discretize the values that $R_1$ and $R_2$ can take to $n=100$ possible values $r_1$, ..., $r_n$, uniformly spaced from $r_1 = -35$ to $r_n = +75$. $R_1$ and $R_2$ are assumed to have Gaussian [marginal distributions](https://en.wikipedia.org/wiki/Marginal_distribution) with known means $\mu_1$ and $\mu_2$ and known standard deviations $\sigma_1$ and $\sigma_2$. Hence, the discretized marginals $p^{(1)}$ and $p^{(2)}$ for $R_1$ and $R_2$ are given by

$$
\displaystyle p_i^{(k)} = \textbf{prob}(R_k = r_i) = \frac{\displaystyle e^{ \frac{-(r_i - \mu_k)^2}{2\sigma_k^2}}}{\displaystyle \sum_{j=1}^n e^{ \frac{-(r_j - \mu_k)^2}{2\sigma_k^2}}}
$$

for $k = 1,2$ and $i = 1, ..., n$. In addition it is known that $R_1$ and $R_2$ are correlated with [correlation coefficient](https://en.wikipedia.org/wiki/Correlation#Pearson's_product-moment_coefficient) $\rho$, which means that

$$
\sum_{i,j} (r_i - \mu_1) \cdot \textbf{prob}(R_1 = r_i , R_2 = r_j) \cdot (r_j - \mu_2) = \rho \sigma_1 \sigma_2
$$


Consider the data $\mu_1 =9$, $\mu_2 = 22$, $\sigma_1 = 7$, $\sigma_2 = 19$ and $\rho = -0.35$.



In [1]:
import cvxpy as cp
import numpy as np

# Data
n = 100
rmin = -35
rmax = 75

mu1 = 9
mu2 = 22
sigma1 = 7
sigma2 = 19
rho = -0.35

In [35]:
R1 = np.linspace(rmin, rmax, n)
R2 = np.linspace(rmin, rmax, n)

def calculate_marginals(r, mu, sigma):
    numerator = np.exp(-(r - mu)**2 / (2*sigma**2))
    denominator = np.sum(numerator)
    return numerator / denominator
    
p1 = calculate_marginals(R1, mu1, sigma1)
p2 = calculate_marginals(R2, mu2, sigma2)

def corr(P): 
    corr = 0
    for i in range(100):
        for j in range(100):
            corr += (R1[i] - mu1) * P[i, j] *  (R2[j] - mu2)
    return corr

P = cp.Variable((n, n))
constraints = [
    cp.sum(P, axis=0) == p1,
    cp.sum(P, axis=1) == p2, 
    corr(P) == rho * sigma1 * sigma2
]

def loss(P):
    p_loss = 0
    for i in range(n):
        for j in range(n):
            if (R1[i] + R2[j] <= 0): 
                p_loss += P[i, j]
    return p_loss

problem = cp.Problem(cp.Maximize(loss(P)), constraints)

problem.solve()
print(P.value)


None


**Formulate the problem as a convex optimisation problem and solve it.**
* (*Extra - IGNORE this sentence unless you finish*) You may compare your result with the case in which $R_1$ and $R_2$ are jointly Gaussian, i.e. $R_1+R_2\sim\mathcal{N}(\mu_1+\mu_2,\sigma_1^2+\sigma_2^2+2\rho\sigma_1\sigma_2)$. In that case the probabilities are uniquely identified by the Gaussian cumulative distribution, so there is no optimisation to do and the loss can be computed directly with a single `scipy.stats` function.

**N.B.** Think. This exercise is not straightforward, but you have one hour to solve it. When in doubt ask for help. Depending on how much time you have left, we will provide help to let all groups reach the solution by the end of the hour.

Below there are two hints. If you click on them, they will expland and you will be able to read them. Use them if you need.

Before starting coding try to write with pen and paper the problem. Start with the constraints and follow with the objective function. If you are unsure about what the variable of the problem is, check Hint 1.


### Hint 1: I do not know what the variable of the problem is

The variable of your problem is $P$, which is the matrix of joint probabilities defined as $P_{ij} = \textbf{prob}(R_1 = r_i, R_2 = r_j)$.

### Hint 2: Help on some constraints

The marginals $p^{(1)}$, $p^{(2)}$ are related to the joint probability matrix $P$ by summation along the two axis of the matrix.

## Exercise 2

Determine whether the following sets are convex or not:

1.   A slab, i.e. a set of the form $\{x \in \mathbb{R}^n : \alpha \le a^\top x \le \beta \}$.
2.   A rectangle, i.e. a set of the form $\{x \in \mathbb{R}^n : \alpha_i \le  x_i \le \beta_i, \, i=1,\dots n \}$.
3.   A wedge, i.e. $\{x \in \mathbb{R}^n : a_1^\top x \le b_1, a_2^\top x \le b_2\}$.
4.   Consider two fixed points $x_0$ and $x_1$. Show that the set $\{x: ||x-x_0||_2 \le ||x-x_1||_2 \}$ is a halfspace.

5.   The set of points closer to a given point than a given set, i.e.$\{ x : ||x-x_0||_2 \le || x - y ||_2 \text{ for all }y\in S\}$ where $S \subseteq{R}^n$

6.   The set of points closer to one set than another, i.e.$\{ x: \textbf{dist}(x,S) \le \textbf{dist}(x,T) \}$ where $S,T \subseteq \mathbb{R}^n$ and $ \textbf{dist}(x,S) = \inf\{||x-z||_2 : z \in S \}.$

7. The set $\{ x : x + S_2 \subseteq S_1 \}$, where $S_1,S_2 \subseteq \mathbb{R}^n$ with $S_1$ convex.



## Exercise 3

Show that if $S_1$ and $S_2$ are convex sets in $\mathbb{R}^{m+n}$, then so is their partial sum

$$
S = \{(x, y_1 + y_2) : x \in \mathbb{R}^m, y_1, y_2 \in \mathbb{R}^n, (x, y_1) \in S_1, (x, y_2) \in S_2\}.
$$